# Notebook 1: Text completion with Oobabooga

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

**Step 1**: Configure your Oobabooga service settings

Use [this notebook](0-AI-settings.ipynb) to save your Oobabooga settings in the configuration file.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs

//Import package for loading hierarchichal settings from settings.json
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4"

#!import config/OobaboogaConnectorConfiguration.cs

// Load configuration using builder package

using Microsoft.Extensions.Configuration;

var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/settings.json", optional: false, reloadOnChange: true);

IConfiguration configuration = builder.Build();

 var oobaboogaConfiguration = configuration.GetSection("Oobabooga").Get<OobaboogaConnectorConfiguration>();

**Step 2**: Import Semantic Kernel SDK and Oobabooga from NuGet

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"
// Import Oobabooga connector
#r "nuget: MyIA.SemanticKernel.Connectors.AI.Oobabooga, 0.31.0"

**Step 3**: Create Oobabooga Text completion settings

In [ ]:
using MyIA.SemanticKernel.Connectors.AI.Oobabooga.Completion.TextCompletion;

//Build settings from configuration file

var oobaboogaTextCompletionSettings = new OobaboogaTextCompletionSettings(
    endpoint: new Uri(oobaboogaConfiguration.EndPoint),
    blockingPort: oobaboogaConfiguration.BlockingPort,
    streamingPort: oobaboogaConfiguration.StreamingPort
);


**Step 4**: Basic Text Completion

In [ ]:
using Microsoft.SemanticKernel.AI.TextCompletion;
 
var inputPrompt = "Once upon a time, ";

var oobaboogaLocal = new OobaboogaTextCompletion(oobaboogaTextCompletionSettings);

 var localResponse = await oobaboogaLocal.CompleteAsync(inputPrompt, new CompleteRequestSettings()
 {
     Temperature = 0.01,
     MaxTokens = 20,
     TopP = 0.1,
 });

 Console.WriteLine(localResponse);

**Step 5**: Streaming Text Completion

In [ ]:
using Microsoft.SemanticKernel.AI.TextCompletion;
 
var inputPrompt = "Once upon a time, ";

var oobaboogaLocal = new OobaboogaTextCompletion(oobaboogaTextCompletionSettings);


var localResponse = oobaboogaLocal.CompleteStreamAsync(inputPrompt, new CompleteRequestSettings()
{
    Temperature = 0.01,
    MaxTokens = 20,
    TopP = 0.1,
});

await foreach (var result in localResponse)
{
    // This will output each piece of the result as it comes in
    Console.Write(result);
}


**Step 6**: Use Kernel with Oobabooga

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.TextCompletion;
using MyIA.SemanticKernel.Connectors.AI.Oobabooga;


//Create Kernel builder
var builder = new KernelBuilder();

builder.WithOobaboogaTextCompletionService(oobaboogaTextCompletionSettings);

IKernel kernel = builder.Build();

// Get text completion service from kernel
var textCompletion = kernel.GetService<ITextCompletion>();
var response = await textCompletion.CompleteAsync("Hello, my name is", new CompleteRequestSettings(){
     Temperature = 0.01,
     MaxTokens = 20,
     TopP = 0.1,
 });
Console.WriteLine(response);

**Step 7**: Load and Run a Skill

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "samples", "skills");

// Load the FunSkill from the Skills Directory
var funSkillFunctions = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

 var contextVariables = new ContextVariables("time travel to dinosaur age");
 contextVariables["style"] = "English humor";
 
 var context = new SKContext(contextVariables);

// Run the Function called Joke
var result = await funSkillFunctions["Joke"].InvokeAsync(context);

// Return the result to the Notebook
Console.WriteLine(result);

**Step 8**: Advanced Text Completion with Custom Chat History

In this section, we'll show you how to simulate a chat experience by maintaining a custom chat history. We'll use text completion to generate responses.

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

// Initialize Kernel Builder
var builder = new KernelBuilder();

// Configure Oobabooga as the AI backend
builder.WithOobaboogaTextCompletionService(oobaboogaTextCompletionSettings);

// Build the Kernel
IKernel kernel = builder.Build();

// Define the prompt
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

// Configure the prompt
var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 100,
        Temperature = 0.7,
        TopP = 0.5,
    }
};

// Register the semantic function
var promptTemplate = new PromptTemplate(skPrompt, promptConfig, kernel);
var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);
var chatFunction = kernel.RegisterSemanticFunction("ChatBot", "Chat", functionConfig);

// Initialize context
var context = kernel.CreateNewContext();
var history = "";
context.Variables["history"] = history;

// Chat function
Func<string, Task> Chat = async (string input) => {
    context.Variables["userInput"] = input;
    var answer = await chatFunction.InvokeAsync(context);
    history += $"\nUser: {input}\nChatBot: {answer}\n";
    context.Variables["history"] = history;
    Console.WriteLine($"ChatBot: {answer}");
};

// Example usage
var message = "Tell me a joke.";
Console.WriteLine($"User: {message}");
await Chat(message);
message = "Why is that funny?";
Console.WriteLine($"User: {message}");
await Chat(message);

This will give you a taste of what's to come in Notebook 2, where we'll dive deeper into chat capabilities.

**Next Steps**: You know the basics, you can now move on to chat capabilities.